![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [122]:
# Run the cells below first

In [109]:
!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [110]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import torch.nn.functional as F

In [111]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())


In [112]:
# Define the CNN class
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # 10 output classes for FashionMNIST

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [113]:
# Define the training loop
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)


In [114]:
# Define DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [115]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [116]:
# Initialize the model, criterion, and optimizer
model = GarmentClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [117]:
# Training the model
num_epochs = 2
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')


Epoch [1/2], Loss: 0.4611
Epoch [2/2], Loss: 0.2916


In [118]:
# Testing the model
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


In [119]:
# Calculate accuracy
accuracy = Accuracy(task='multiclass', num_classes=10)(torch.tensor(predictions), torch.tensor(true_labels)).item()



In [120]:
precision = Precision(task='multiclass', average=None, num_classes=10)(torch.tensor(predictions), torch.tensor(true_labels)).tolist()
recall = Recall(task='multiclass', average=None, num_classes=10)(torch.tensor(predictions), torch.tensor(true_labels)).tolist()
print(f'Accuracy: {accuracy:.4f}')
print(f'Per-class Precision: {precision}')
print(f'Per-class Recall: {recall}')


Accuracy: 0.8907
Per-class Precision: [0.8900584578514099, 0.980079710483551, 0.79654860496521, 0.8432971239089966, 0.8558456301689148, 0.9848331809043884, 0.6913461685180664, 0.9299242496490479, 0.9693675637245178, 0.9853861927986145]
Per-class Recall: [0.7609999775886536, 0.984000027179718, 0.8769999742507935, 0.9309999942779541, 0.7540000081062317, 0.9739999771118164, 0.718999981880188, 0.9819999933242798, 0.9810000061988831, 0.9440000057220459]
